In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

In [2]:
import os # 모델 저장할 폴더가 있는지 여부와 폴더 생성
import numpy as np
import pandas as pd # get_dummies(원핫인코딩), crosstab
import seaborn as sns # iris 데이터(데이터프레임) 가져오기
from sklearn import datasets # iris 데이터(X, y) 가져오기
from sklearn.preprocessing import LabelEncoder # 라벨인코더
from tensorflow.keras.utils import to_categorical # 원핫인코딩 
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras import metrics # compile
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

# 1. 기본적인 DNN

In [9]:
# 1. 데이터 생성 및 전처리
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target.reshape(-1, 1)
train_X, test_X, train_y, test_y = train_test_split(iris_X, iris_y,
                                                   test_size=0.2,
                                                   stratify=iris_y)
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120, 1), (30, 4), (30, 1))

In [4]:
# 2. 모델구성
model = Sequential([
    Input(4),
    Dense(units=64, activation='relu'),
    Dense(units=128, activation='relu'),
    Dropout(0.2), 
    Dense(units=50, activation='relu'),
    Dropout(0.2),
    Dense(units=30, activation='relu'),
    Dropout(0.1),
    Dense(units=3, activation='softmax')    
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 50)                6450      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_3 (Dense)             (None, 30)                1530      
                                                                 
 dropout_2 (Dropout)         (None, 30)                0

In [7]:
# 3. 학습과정 설정
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy', metrics.Recall(), metrics.Precision()])
# 4. 모델 학습
earlystopping = EarlyStopping(patience=40)# monitor='val_loss' 기본값
hist=model.fit(train_X,train_y,batch_size=40,
               epochs=200,
               validation_split=0.2,
               callbacks=[earlystopping])

Epoch 1/200


ValueError: in user code:

    File "C:\Users\901-00\anaconda3\envs\ml-dl-nlp\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\901-00\anaconda3\envs\ml-dl-nlp\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\901-00\anaconda3\envs\ml-dl-nlp\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\901-00\anaconda3\envs\ml-dl-nlp\lib\site-packages\keras\engine\training.py", line 998, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "C:\Users\901-00\anaconda3\envs\ml-dl-nlp\lib\site-packages\keras\engine\training.py", line 1092, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "C:\Users\901-00\anaconda3\envs\ml-dl-nlp\lib\s